In [1]:
%matplotlib inline
import xarray as xr
import datetime as dt
from datetime import timedelta
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.readers import reader_global_landmask
from opendrift.models.oceandrift import OceanDrift
import opendrift
import os
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('TkAgg')

In [2]:
opendrift.readers.reader_global_landmask.__file__

'/home/pdaniel/anaconda3/lib/python3.8/site-packages/OpenDrift-1.3.2-py3.8.egg/opendrift/readers/reader_global_landmask.py'

In [3]:
def make_map():
    fig = plt.figure(figsize=(12,9))
    cart_proj = ccrs.PlateCarree()
    ax = plt.axes(projection=cart_proj)
    ax.coastlines('10m', linewidth=0.8,zorder=200)
    ax.set_xlim(-123.3,-122.2)
    ax.set_ylim(37.5,38.1)
    return(fig, ax)

## Model Setup ##

__1. Create the Model Object__

We will be using the OceanDrift model, which simulates ocean surface drifting, forced by surface currents, wind, and/or stokes drift.

In [8]:
o = OceanDrift(loglevel=50) # Setting the log level will give the user different amounts of info (0=Debug (all), 20=Minimum, 50=None)
reader_2km = reader_netCDF_CF_generic.Reader("http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd")
reader_6km = reader_netCDF_CF_generic.Reader("http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd")
reader_landmask = reader_global_landmask.Reader(
                       extent=[-123.99,  -122.177032, 37.244221, 38.233120])  # lonmin, lonmax, latmin, latmax
o.add_reader([reader_landmask,reader_2km, reader_6km])



In [9]:
reader_landmask = reader_global_landmask.Reader(
                       extent=[-123.99,  -122.177032, 37.244221, 38.233120])  # lonmin, lonmax, latmin, latmax

ERROR:root:could not verify read permissions for group and others on landmask.
Traceback (most recent call last):
  File "/home/pdaniel/anaconda3/lib/python3.8/site-packages/opendrift_landmask_data-0.6-py3.8.egg/opendrift_landmask_data/mask.py", line 77, in __check_permissions__
    if not os.stat(self.lockf).st_mode & 0o777 == 0o777:
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/landmask/.mask.dat.lock'


__3. Add readers to model__

Data readers can be added in an hierarchical manner, from highest to lowest prioity.

In [10]:
o.add_reader([reader_landmask,reader_2km, reader_6km])
# o.add_reader([reader_2km, reader_6km])


o

Model:	OceanDrift     (OpenDrift version 1.3.2)
	0 active Lagrangian3DArray particles  (0 deactivated, 0 scheduled)
Projection: +proj=lonlat +ellps=WGS84
-------------------
Environment variables:
  -----
  land_binary_mask
     1) global_landmask
  -----
  x_sea_water_velocity
  y_sea_water_velocity
     1) http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd
     2) http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd
  -----
Readers not added for the following variables:
  ocean_vertical_diffusivity
  sea_floor_depth_below_sea_level
  sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment
  sea_surface_wave_period_at_variance_spectral_density_maximum
  sea_surface_wave_significant_height
  sea_surface_wave_stokes_drift_x_velocity
  sea_surface_wave_stokes_drift_y_velocity
  surface_downward_x_stress
  surface_downward_

__4. Seeding the Model__

There are a several different ways to seed data, we can supply a shape file, a radius, a line etc.

From the tutorial:
```
Note that the radius is not an absolute boundary within which elements will be seeded, but one standard deviation of a normal distribution in space. Thus about 68% of elements will be seeded within this radius, with more elements near the center. By default, elements are seeded at the surface (z=0)
```

Here we could also use the OilDrift model if we wanted to simulate oils (or other density fluids).

In [ ]:
# 2020-12-01 14:00:00
# 37.8016, -122.531
start_time = dt.datetime(2020,12,1,21,0)
time_step = timedelta(hours=1)
num_steps = 24

for i in range(num_steps+1):
    o.seed_elements(lon=-122.531, lat=37.8016, number=20, radius=500,
            time=start_time + i*time_step)

__5. Setting other configurations__

There are a bunch of ways to configure the model, how it reflects off of the shoreline, the advection schemes, other forcings etc.

Use `o.list_configspec()` to view the current settings

In [ ]:
o.set_config('general:coastline_action', 'stranding') 
o.set_config('drift:scheme', 'runge-kutta4')
o.set_config('general:time_step_minutes', 15)
o.set_config('drift:current_uncertainty_uniform', .5) # uncertainty .2 meters per sec. this is a uniform distribution from -.2 to .2

o.set_config('drift:stokes_drift', False)

In [ ]:
o.list_environment_variables()

In [ ]:
base_folder= "Data/model_output/"
out_fname = "sf_bay_continuous_20_gate_" + start_time.strftime("%Y%m%dT%H%M%S")

In [ ]:
os.path.join(base_folder,out_fname+".nc")

In [ ]:
o.run(duration=timedelta(hours=48), time_step=timedelta(minutes=15), outfile=os.path.join(base_folder,out_fname+".nc")) #
# o.run(duration=timedelta(hours=48), time_step=timedelta(minutes=15)) #

In [ ]:
o.animation(legend_loc='upper center',background=["x_sea_water_velocity","y_sea_water_velocity"], fast=True,  filename='Figures/continuous-start.mp4',show_trajectories=True)

In [ ]:
# o.plot(show_particles=True,background="x_sea_water_velocity")
# o.plot(compare=o_diffusion, legend=['Width diffusion', 'No diffusion'],density=True)
o.plot(density=True, filename='Figures/continuos_plot.png')
plt.show()

In [ ]:
# color='wind_drift_factor'
o.plot(linecolor='age_seconds')

# print(o)

In [ ]:
o_diffusion.write_netcdf_density_map('Data/model_output/diffusion_density_large.nc',pixelsize_m=1000)

In [ ]:
o_diffusion.

In [ ]:
o.reset()
o_diffusion.reset()

In [ ]:
import opendrift

In [ ]:
opendrift.__version__